<a href="https://colab.research.google.com/github/grissomlin/US-Leads-Scraper-Pro/blob/main/%E7%BE%8E%E5%9C%8B%E7%8D%B2%E5%AE%A2%E7%A8%8B%E5%BC%8F_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from textwrap import dedent

BASE_DIR = "/content"
os.chdir(BASE_DIR)

prompts_code = dedent(r'''# -*- coding: utf-8 -*-
"""
prompts.py（產業無關通用版 + UI 版支援）

✅ 話術 / Prompt 中央設定檔
✅ 支援：
   - 固定預設（CompanyProfile）
   - Sidebar UI 即時覆蓋（ui_profile dict）
   - 多語言（English / Spanish / French）
✅ 預設內容為「美甲設備外貿（示範）」，可自行更換任何產業

-----------------------------
用法：
from prompts import SalesPrompts, DEFAULT_PROFILE
-----------------------------
"""

from __future__ import annotations
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional


# ============================================================
# 1) CompanyProfile：預設公司設定（示範值，可自行修改）
# ============================================================
@dataclass
class CompanyProfile:
    """
    👉 這裡是「預設公司介紹」
    👉 不用 UI 的情況下，系統就會用這組
    """

    COMPANY_NAME: str = "Nail Equipment Export Co., Ltd. (Demo)"
    YOUR_NAME: str = "[Your Name]"
    CONTACT_PHONE: str = "(888) XXX-XXXX"
    CONTACT_EMAIL: str = "contact@example.com"
    COMPANY_WEBSITE: str = "https://www.example.com"

    BUSINESS_TYPE: str = "Factory-direct exporter of professional salon equipment (Demo)"

    DEFAULT_PRODUCTS: List[str] = field(
        default_factory=lambda: [
            "High-speed electric nail drills (ceramic / steel bits)",
            "Professional dust collectors & ventilation systems",
            "Pedicure chairs for salons",
            "UV / LED curing lamps & sterilization equipment",
        ]
    )

    VALUE_PROPOSITIONS: List[str] = field(
        default_factory=lambda: [
            "Factory-direct pricing (save 30–50% vs distributors)",
            "US warehouse stock with fast 3–5 day delivery",
            "Low MOQ for first-time cooperation",
            "OEM / private label support",
            "Lifetime technical support",
        ]
    )


# ✅ 模組層級的預設 profile（讓 app_lead.py 可以 import）
DEFAULT_PROFILE = CompanyProfile()


# ============================================================
# 2) SalesPrompts：銷售話術生成器
# ============================================================
class SalesPrompts:
    # -------------------------
    # 共用工具
    # -------------------------
    @staticmethod
    def _clean_lines(items: Optional[List[str]]) -> List[str]:
        if not items:
            return []
        out: List[str] = []
        for x in items:
            s = "" if x is None else str(x).strip()
            if s:
                out.append(s)
        return out

    @staticmethod
    def _bullet_lines(items: List[str]) -> str:
        items = SalesPrompts._clean_lines(items)
        if not items:
            return "• (Not specified)"
        return "\n".join([f"• {x}" for x in items])

    # -------------------------
    # 語言解析（強化版）
    # -------------------------
    @staticmethod
    def _get_lang(ui_profile: Optional[Dict[str, Any]]) -> str:
        if not ui_profile:
            return "English"
        lang_raw = str(ui_profile.get("LANG", "English")).strip().lower()

        if lang_raw in ("en", "english"):
            return "English"
        if lang_raw in ("es", "spanish", "español", "espanol"):
            return "Spanish"
        if lang_raw in ("fr", "french", "français", "francais"):
            return "French"

        # fallback: prefix match
        if lang_raw.startswith("en"):
            return "English"
        if lang_raw.startswith("es"):
            return "Spanish"
        if lang_raw.startswith("fr"):
            return "French"
        return "English"

    # ============================================================
    # 固定版（不用 UI）
    # ============================================================
    @staticmethod
    def generate_sales_email(
        business_name: str,
        city: str,
        state: str,
        products: Optional[List[str]] = None,
        lang: str = "English",
        profile: CompanyProfile = DEFAULT_PROFILE,
    ) -> str:
        products_list = products if products is not None else profile.DEFAULT_PRODUCTS
        values_list = profile.VALUE_PROPOSITIONS

        product_lines = SalesPrompts._bullet_lines(products_list)
        value_lines = SalesPrompts._bullet_lines(values_list)

        lang_norm = "English"
        if str(lang).lower().startswith("es"):
            lang_norm = "Spanish"
        elif str(lang).lower().startswith("fr"):
            lang_norm = "French"

        if lang_norm == "Spanish":
            return f"""
Hola equipo de {business_name} en {city}, {state},

Encontramos su negocio y nos gustaría presentarnos brevemente.

🏢 Quiénes somos:
{profile.BUSINESS_TYPE}

📦 Productos / Servicios:
{product_lines}

⭐ Por qué trabajar con nosotros:
{value_lines}

¿Le interesaría recibir un catálogo o agendar una llamada breve de 10–15 minutos?

Saludos,
{profile.YOUR_NAME}
{profile.COMPANY_NAME}
📞 {profile.CONTACT_PHONE}
📧 {profile.CONTACT_EMAIL}
🌐 {profile.COMPANY_WEBSITE}
""".strip()

        if lang_norm == "French":
            return f"""
Bonjour à l'équipe de {business_name} à {city}, {state},

Nous avons découvert votre entreprise et souhaitons nous présenter brièvement.

🏢 Qui nous sommes :
{profile.BUSINESS_TYPE}

📦 Produits / Services :
{product_lines}

⭐ Pourquoi travailler avec nous :
{value_lines}

Seriez-vous intéressé(e) par un catalogue ou un échange de 10–15 minutes ?

Cordialement,
{profile.YOUR_NAME}
{profile.COMPANY_NAME}
📞 {profile.CONTACT_PHONE}
📧 {profile.CONTACT_EMAIL}
🌐 {profile.COMPANY_WEBSITE}
""".strip()

        return f"""
Dear {business_name} team in {city}, {state},

We came across your business and would like to briefly introduce our company.

🏢 Who we are:
{profile.BUSINESS_TYPE}

📦 Our products / services:
{product_lines}

⭐ Why work with us:
{value_lines}

Would you be open to receiving a catalog or scheduling a short 10–15 minute call?

Best regards,
{profile.YOUR_NAME}
{profile.COMPANY_NAME}
📞 {profile.CONTACT_PHONE}
📧 {profile.CONTACT_EMAIL}
🌐 {profile.COMPANY_WEBSITE}
""".strip()

    # ============================================================
    # UI 版（Sidebar 即時覆蓋）
    # ============================================================
    @staticmethod
    def generate_sales_email_ui(
        business_name: str,
        city: str,
        state: str,
        products: Optional[List[str]],
        ui_profile: Dict[str, Any],
    ) -> str:
        lang = SalesPrompts._get_lang(ui_profile)

        company = str(ui_profile.get("COMPANY_NAME", DEFAULT_PROFILE.COMPANY_NAME)).strip()
        biz = str(ui_profile.get("BUSINESS_TYPE", DEFAULT_PROFILE.BUSINESS_TYPE)).strip()
        your = str(ui_profile.get("YOUR_NAME", DEFAULT_PROFILE.YOUR_NAME)).strip()
        phone = str(ui_profile.get("CONTACT_PHONE", DEFAULT_PROFILE.CONTACT_PHONE)).strip()
        email = str(ui_profile.get("CONTACT_EMAIL", DEFAULT_PROFILE.CONTACT_EMAIL)).strip()
        web = str(ui_profile.get("COMPANY_WEBSITE", DEFAULT_PROFILE.COMPANY_WEBSITE)).strip()

        values = ui_profile.get("VALUE_PROPOSITIONS", DEFAULT_PROFILE.VALUE_PROPOSITIONS)
        default_products = ui_profile.get("DEFAULT_PRODUCTS", DEFAULT_PROFILE.DEFAULT_PRODUCTS)

        values_list = SalesPrompts._clean_lines(values if isinstance(values, list) else DEFAULT_PROFILE.VALUE_PROPOSITIONS)
        products_list = (
            SalesPrompts._clean_lines(products)
            if products is not None
            else SalesPrompts._clean_lines(default_products if isinstance(default_products, list) else DEFAULT_PROFILE.DEFAULT_PRODUCTS)
        )

        product_lines = SalesPrompts._bullet_lines(products_list)
        value_lines = SalesPrompts._bullet_lines(values_list)

        if lang == "Spanish":
            return f"""
Hola equipo de {business_name} en {city}, {state},

Encontramos su negocio y nos gustaría presentarnos brevemente.

🏢 Quiénes somos:
{biz}

📦 Productos / Servicios:
{product_lines}

⭐ Por qué trabajar con nosotros:
{value_lines}

¿Le interesaría recibir un catálogo o agendar una llamada breve de 10–15 minutos?

Saludos,
{your}
{company}
📞 {phone}
📧 {email}
🌐 {web}
""".strip()

        if lang == "French":
            return f"""
Bonjour à l'équipe de {business_name} à {city}, {state},

Nous avons découvert votre entreprise et souhaitons nous présenter brièvement.

🏢 Qui nous sommes :
{biz}

📦 Produits / Services :
{product_lines}

⭐ Pourquoi travailler avec nous :
{value_lines}

Seriez-vous intéressé(e) par un catalogue ou un échange de 10–15 minutes ?

Cordialement,
{your}
{company}
📞 {phone}
📧 {email}
🌐 {web}
""".strip()

        return f"""
Dear {business_name} team in {city}, {state},

We came across your business and would like to briefly introduce our company.

🏢 Who we are:
{biz}

📦 Our products / services:
{product_lines}

⭐ Why work with us:
{value_lines}

Would you be open to receiving a catalog or scheduling a short 10–15 minute call?

Best regards,
{your}
{company}
📞 {phone}
📧 {email}
🌐 {web}
""".strip()

    # ============================================================
    # UI 顯示用：提示詞模板（不進 LLM）
    # ============================================================
    @staticmethod
    def prompt_templates_markdown_ui(ui_profile: Dict[str, Any]) -> str:
        lang = SalesPrompts._get_lang(ui_profile)

        company = str(ui_profile.get("COMPANY_NAME", DEFAULT_PROFILE.COMPANY_NAME)).strip()
        biz = str(ui_profile.get("BUSINESS_TYPE", DEFAULT_PROFILE.BUSINESS_TYPE)).strip()
        products = ui_profile.get("DEFAULT_PRODUCTS", DEFAULT_PROFILE.DEFAULT_PRODUCTS)
        values = ui_profile.get("VALUE_PROPOSITIONS", DEFAULT_PROFILE.VALUE_PROPOSITIONS)

        products_list = SalesPrompts._clean_lines(products if isinstance(products, list) else DEFAULT_PROFILE.DEFAULT_PRODUCTS)
        values_list = SalesPrompts._clean_lines(values if isinstance(values, list) else DEFAULT_PROFILE.VALUE_PROPOSITIONS)

        p = "\n".join([f"• {x}" for x in products_list]) if products_list else "• (Not specified)"
        v = "\n".join([f"• {x}" for x in values_list]) if values_list else "• (Not specified)"

        if lang == "Spanish":
            return f"""
PROMPT TEMPLATE (ES)

Soy {company} ({biz}).
Quiero contactar a [Nombre del negocio] en [Ciudad, Estado].

Productos/Servicios:
{p}

Ventajas:
{v}

Escribe un email B2B corto, profesional y amable.
Pide si desean un catálogo/lista de precios o una llamada de 10–15 minutos.
""".strip()

        if lang == "French":
            return f"""
PROMPT TEMPLATE (FR)

Je suis {company} ({biz}).
Je souhaite contacter [Nom de l'entreprise] à [Ville, État].

Produits/Services :
{p}

Avantages :
{v}

Rédige un email B2B court, professionnel et cordial.
Propose l'envoi d'un catalogue/grille tarifaire ou un échange de 10–15 minutes.
""".strip()

        return f"""
PROMPT TEMPLATE (EN)

I am {company} ({biz}).
I want to reach out to [Business Name] in [City, State].

Products/Services:
{p}

Advantages:
{v}

Write a short, professional, and polite B2B outreach email.
Ask if they want a catalog/pricing sheet or a 10–15 minute call.
""".strip()
''')

with open("/content/prompts.py", "w", encoding="utf-8") as f:
    f.write(prompts_code)

print("✅ prompts.py 已更新：/content/prompts.py（包含 DEFAULT_PROFILE）")


In [ ]:
import os, zipfile
from textwrap import dedent

# ================
# 0) 基本路徑
# ================
BASE_DIR = "/content"
os.chdir(BASE_DIR)
print("✅ cwd =", os.getcwd())

# ================
# 1) 安裝套件
# ================
!pip -q install google-colab-selenium selenium-stealth pandas openpyxl streamlit streamlit_folium folium pyngrok

# ================
# 2) 解壓 uszips.csv（若尚未存在）
# ================
ZIP_NAME = "simplemaps_uszips_basicv1.80.zip"
if not os.path.exists("uszips.csv"):
    if os.path.exists(ZIP_NAME):
        with zipfile.ZipFile(ZIP_NAME, "r") as z:
            z.extractall(".")
        print("✅ 已解壓 uszips.csv")
    else:
        print("⚠️ 找不到 ZIP 檔，請先把 simplemaps_uszips_basicv1.80.zip 放到 /content")

# ================
# 3) 檢查 prompts.py 是否存在（不再生成、不覆蓋）
# ================
if os.path.exists("prompts.py"):
    print("✅ prompts.py 已存在（將由 app_lead.py 直接引用）")
else:
    print("⚠️ prompts.py 不存在，請先建立公司自我介紹/提示詞檔案")
    print("👉 你可以自行建立 prompts.py，或我可以幫你產一個『通用範本版』")


In [ ]:
# 這個單元格只寫入 worker.py 文件，不執行任何代碼

worker_code = '''# -*- coding: utf-8 -*-

"""
Google Maps Leads Scraper (Worker)
- Reads: dashboard_started.flag + config.json (from Streamlit dashboard)
- Writes: scan_progress.json + leads.csv + leads.xlsx
- Focus: improve hit-rate for Name / Phone / Website / Address
"""

import os, sys, json, time, random, re
from datetime import datetime
import urllib.parse

import pandas as pd

import google_colab_selenium as gs
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium_stealth import stealth


# =========================
# Paths
# =========================
def get_base_dir():
    try:
        return os.path.dirname(os.path.abspath(__file__))
    except Exception:
        return os.getcwd()


# =========================
# Helpers
# =========================
def now_str():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def update_progress(current_zip, found, scanned, status="RUNNING", log_msg="", limit=0):
    BASE_DIR = get_base_dir()
    PROG_PATH = os.path.join(BASE_DIR, "scan_progress.json")
    payload = {
        "current_zip": str(current_zip),
        "found_count": int(found),
        "total_scanned": int(scanned),
        "limit": int(limit),
        "status": status,
        "log": str(log_msg),
        "timestamp": now_str(),
    }
    try:
        with open(PROG_PATH, "w", encoding="utf-8") as f:
            json.dump(payload, f, ensure_ascii=False)
    except Exception as e:
        print("[WARN] progress write failed:", e)

def normalize_zip(z):
    s = str(z).strip()
    s = re.sub(r"\\.0$", "", s)
    s = re.sub(r"\\D", "", s)
    return s.zfill(5) if s else ""

def _clean_text(s: str) -> str:
    return (s or "").replace("\\u200b", "").strip()

def _first_nonempty(*vals):
    for v in vals:
        v = _clean_text(v)
        if v:
            return v
    return ""

def _maybe_prefix_http(url: str) -> str:
    url = _clean_text(url)
    if not url:
        return ""
    if url.startswith("www."):
        return "http://" + url
    return url

def extract_phone(text: str) -> str:
    if not text:
        return ""
    patterns = [
        r"(?:\\+?1[-.\\s]?)?\\(?\\d{3}\\)?[-.\\s]?\\d{3}[-.\\s]?\\d{4}",
        r"\\d{10}",
    ]
    for pat in patterns:
        m = re.search(pat, text)
        if m:
            return m.group(0).strip()
    return ""

def extract_url(text: str) -> str:
    if not text:
        return ""
    m = re.search(r"https?://[^\\s]+", text)
    if m:
        return m.group(0).strip()
    m = re.search(r"www\\.[^\\s]+", text)
    if m:
        return "http://" + m.group(0).strip()
    return ""

def handle_consent_if_any(driver):
    """Best-effort handle consent popups."""
    try:
        time.sleep(0.8)
        candidates = driver.find_elements(By.CSS_SELECTOR, "button, div[role='button']")
        for b in candidates[:50]:
            t = _clean_text(getattr(b, "text", "")).lower()
            if any(k in t for k in ["accept all", "i agree", "agree", "accept", "同意", "接受全部"]):
                try:
                    driver.execute_script("arguments[0].click();", b)
                except Exception:
                    try:
                        b.click()
                    except Exception:
                        pass
                time.sleep(1.2)
                return
    except Exception:
        pass

def get_place_name(driver, anchor):
    """改進的店名提取函數 - 多種方法獲取有效店名"""
    name = ""

    # 方法1: 從 anchor 的 aria-label 獲取
    try:
        name = _clean_text(anchor.get_attribute("aria-label"))
        # 檢查是否為有效店名（不是 "RESULT", "結果" 等）
        if name and len(name) > 2 and not any(keyword in name.lower() for keyword in ["result", "results", "search result", "search results", "搜尋結果", "結果"]):
            print(f"[get_place_name] 從 aria-label 獲取: {name}")
            return name
    except Exception:
        pass

    # 方法2: 從卡片中的店名元素獲取
    try:
        # 找尋卡片中的店名元素 (Google Maps 常見選擇器)
        card = anchor.find_element(By.XPATH, "./ancestor::div[contains(@class, 'Nv2PK')]")
        name_selectors = [
            "div[class*='fontHeadlineSmall']",
            "div[class*='fontHeadlineMedium']",
            "div[role='heading']",
            "h3",
            "div.qBF1Pd"
        ]

        for selector in name_selectors:
            try:
                elements = card.find_elements(By.CSS_SELECTOR, selector)
                for elem in elements:
                    name = _clean_text(elem.text)
                    if name and len(name) > 2 and not any(keyword in name.lower() for keyword in ["result", "results"]):
                        print(f"[get_place_name] 從選擇器 {selector} 獲取: {name}")
                        return name
            except Exception:
                continue
    except Exception:
        pass

    # 方法3: 從 anchor 文本獲取
    try:
        name = _clean_text(anchor.text)
        if name and len(name) > 2 and not any(keyword in name.lower() for keyword in ["result", "results"]):
            print(f"[get_place_name] 從 anchor 文本獲取: {name}")
            return name
    except Exception:
        pass

    # 方法4: 從卡片中提取第一個有意義的文本
    try:
        card = anchor.find_element(By.XPATH, "./ancestor::div[contains(@class, 'Nv2PK')]")
        all_text = _clean_text(card.text)
        lines = all_text.split("\\n")
        for line in lines:
            line = line.strip()
            if line and len(line) > 2 and not any(keyword in line.lower() for keyword in ["result", "results", "directions", "save", "分享"]):
                name = line
                print(f"[get_place_name] 從卡片文本獲取: {name}")
                return name
    except Exception:
        pass

    print(f"[get_place_name] 無法獲取有效店名，返回空值")
    return ""

def extract_from_detail_panel(driver):
    """改進的詳細信息提取 - 多種選擇器提高成功率"""
    phone = ""
    address = ""
    website = ""

    try:
        # 等待詳細面板加載
        time.sleep(2)

        print("[extract_from_detail_panel] 開始提取詳細信息")

        # 電話號碼提取 - 多種選擇器
        phone_selectors = [
            "button[data-item-id*='phone']",
            "button[data-tooltip*='phone']",
            "button[aria-label*='phone']",
            "div[data-tooltip*='phone']",
            "a[href^='tel:']",
            "//button[contains(@aria-label, 'phone') or contains(@aria-label, 'call')]",
            "//div[contains(@aria-label, 'phone') or contains(@aria-label, 'call')]",
            "//*[contains(text(), '(') and contains(text(), ')') and string-length(text()) < 20]",
            "//*[contains(text(), '-') and string-length(text()) > 9 and string-length(text()) < 20]"
        ]

        for selector in phone_selectors:
            try:
                if selector.startswith("//"):
                    elements = driver.find_elements(By.XPATH, selector)
                else:
                    elements = driver.find_elements(By.CSS_SELECTOR, selector)

                for elem in elements:
                    text = _clean_text(elem.text)
                    if text:
                        # 先檢查文本是否包含電話號碼
                        phone_match = extract_phone(text)
                        if phone_match and len(phone_match) >= 10:
                            phone = phone_match
                            print(f"[extract_from_detail_panel] 從選擇器 {selector} 找到電話: {phone}")
                            break
                        # 如果文本看起來像電話號碼但不是標準格式
                        elif re.search(r"\\d{3}[-.]?\\d{3}[-.]?\\d{4}", text):
                            phone = re.search(r"\\d{3}[-.]?\\d{3}[-.]?\\d{4}", text).group()
                            print(f"[extract_from_detail_panel] 從選擇器 {selector} 找到電話(非標準): {phone}")
                            break
                if phone:
                    break
            except Exception as e:
                continue

        # 如果還沒有找到，嘗試從整個頁面提取電話
        if not phone:
            try:
                page_text = driver.find_element(By.TAG_NAME, "body").text
                phone_match = extract_phone(page_text)
                if phone_match:
                    phone = phone_match
                    print(f"[extract_from_detail_panel] 從頁面文本找到電話: {phone}")
            except Exception:
                pass

        # 地址提取
        address_selectors = [
            "button[data-item-id*='address']",
            "button[data-tooltip*='address']",
            "button[aria-label*='address']",
            "div[data-tooltip*='address']",
            "//button[contains(@aria-label, 'address')]",
            "//div[contains(@aria-label, 'address')]",
            "//*[contains(text(), 'Street') or contains(text(), 'St') or contains(text(), 'Ave') or contains(text(), 'Avenue') or contains(text(), 'Rd') or contains(text(), 'Road') or contains(text(), 'Blvd')]"
        ]

        for selector in address_selectors:
            try:
                if selector.startswith("//"):
                    elements = driver.find_elements(By.XPATH, selector)
                else:
                    elements = driver.find_elements(By.CSS_SELECTOR, selector)

                for elem in elements:
                    text = _clean_text(elem.text)
                    if text and len(text) > 10 and not any(x in text.lower() for x in ["phone", "call", "website", "web", "方向", "導航"]):
                        address = text
                        print(f"[extract_from_detail_panel] 從選擇器 {selector} 找到地址: {address}")
                        break
                if address:
                    break
            except Exception:
                continue

        # 網站提取
        website_selectors = [
            "a[data-item-id*='authority']",
            "a[aria-label*='website']",
            "a[href*='http']:not([href*='google']):not([href*='maps']):not([href*='facebook'])",
            "//a[contains(@href, 'http') and not(contains(@href, 'google')) and not(contains(@href, 'maps'))]",
            "button[aria-label*='website']",
            "button[data-tooltip*='website']"
        ]

        for selector in website_selectors:
            try:
                if selector.startswith("//"):
                    elements = driver.find_elements(By.XPATH, selector)
                else:
                    elements = driver.find_elements(By.CSS_SELECTOR, selector)

                for elem in elements:
                    # 對於按鈕，檢查文本
                    if elem.tag_name == "button":
                        text = _clean_text(elem.text)
                        url_match = extract_url(text)
                        if url_match:
                            website = url_match
                            print(f"[extract_from_detail_panel] 從按鈕文本找到網站: {website}")
                            break
                    # 對於鏈接，檢查 href 屬性
                    else:
                        href = _clean_text(elem.get_attribute("href"))
                        if href and not any(x in href for x in ["google.com", "goo.gl", "maps.google", "facebook.com", "twitter.com"]):
                            website = href
                            print(f"[extract_from_detail_panel] 從 href 找到網站: {website}")
                            break
                if website:
                    break
            except Exception:
                continue

        # 如果還沒有找到網站，嘗試從按鈕文本中提取
        if not website:
            try:
                buttons = driver.find_elements(By.CSS_SELECTOR, "button")
                for btn in buttons:
                    text = _clean_text(btn.text)
                    url_match = extract_url(text)
                    if url_match:
                        website = url_match
                        print(f"[extract_from_detail_panel] 從按鈕文本找到網站(備用): {website}")
                        break
            except Exception:
                pass

    except Exception as e:
        print(f"[extract_from_detail_panel] 錯誤: {e}")

    return {
        "Phone": phone,
        "Address": address,
        "Website": _maybe_prefix_http(website),
    }

def extract_business_info(driver, anchor, card_text=""):
    """改進的商業信息提取 - 增加錯誤處理和調試信息"""
    print(f"[extract_business_info] 開始提取商業信息")

    # 先獲取店名
    name = get_place_name(driver, anchor)

    # 如果店名無效，直接返回
    if not name or len(name) < 2 or any(keyword in name.lower() for keyword in ["result", "results", "search result"]):
        print(f"[extract_business_info] 無效店名: {name}，跳過")
        return None

    print(f"[extract_business_info] 處理商家: {name}")

    # 點擊進入詳情頁
    try:
        # 保存當前 URL 以便返回
        original_url = driver.current_url

        # 嘗試點擊進入詳情頁
        try:
            driver.execute_script("arguments[0].click();", anchor)
        except Exception:
            try:
                anchor.click()
            except Exception:
                print(f"[extract_business_info] 點擊失敗，嘗試通過 href 訪問")
                try:
                    href = anchor.get_attribute("href")
                    if href:
                        driver.get(href)
                except Exception:
                    print(f"[extract_business_info] 無法訪問商家詳情頁")
                    return None

        # 等待詳情頁加載
        time.sleep(3)

        # 處理可能的同意彈窗
        handle_consent_if_any(driver)

        # 提取詳細信息
        detail = extract_from_detail_panel(driver)

        # 返回結果列表頁
        try:
            driver.back()
            time.sleep(2)
        except Exception:
            # 如果返回失敗，嘗試返回原始 URL
            try:
                driver.get(original_url)
                time.sleep(2)
            except Exception:
                pass

    except Exception as e:
        print(f"[extract_business_info] 提取過程中出錯: {e}")
        # 嘗試返回原始頁面
        try:
            driver.back()
            time.sleep(2)
        except Exception:
            pass
        return None

    # 合併所有文本進行後續提取
    merged_text = f"{card_text}\\n{detail.get('Phone', '')}\\n{detail.get('Address', '')}\\n{detail.get('Website', '')}"

    # 優先使用詳情頁提取的，如果沒有則嘗試從合併文本中提取
    phone = detail.get("Phone", "")
    if not phone:
        phone = extract_phone(merged_text)

    website = detail.get("Website", "")
    if not website:
        website = extract_url(merged_text)

    address = detail.get("Address", "")
    if not address:
        # 嘗試從卡片文本中提取地址
        address_match = re.search(r"\\d+[\\s\\w.,-]+(?:Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd)[\\s\\w.,-]*", card_text, re.IGNORECASE)
        if address_match:
            address = address_match.group(0).strip()

    # 返回結果
    result = {
        "Name": name,
        "Phone": phone or "N/A",
        "Address": address or "N/A",
        "Website": _maybe_prefix_http(website) or "N/A",
    }

    print(f"[extract_business_info] 提取結果: {result}")
    return result


# =========================
# Main
# =========================
def main():
    BASE_DIR = get_base_dir()
    FLAG_PATH = os.path.join(BASE_DIR, "dashboard_started.flag")
    CONFIG_PATH = os.path.join(BASE_DIR, "config.json")
    PROG_PATH = os.path.join(BASE_DIR, "scan_progress.json")
    ZIPS_PATH = os.path.join(BASE_DIR, "uszips.csv")
    LEADS_CSV = os.path.join(BASE_DIR, "leads.csv")
    LEADS_XLSX = os.path.join(BASE_DIR, "leads.xlsx")

    print("=" * 60)
    print("[worker] 開始執行")
    print("BASE_DIR =", BASE_DIR)
    print("cwd =", os.getcwd())
    print("FLAG 存在?", os.path.exists(FLAG_PATH))
    print("CONFIG 存在?", os.path.exists(CONFIG_PATH))
    print("=" * 60)

    # 等待 flag (最多 30 秒)
    waited = 0.0
    while not os.path.exists(FLAG_PATH) and waited < 30.0:
        time.sleep(0.5)
        waited += 0.5
        if waited % 5 == 0:
            print(f"[worker] 等待儀表板啟動信號... ({waited}/30 秒)")

    if not os.path.exists(FLAG_PATH):
        msg = f"請從儀表板啟動 worker | cwd={os.getcwd()} | base={BASE_DIR}"
        print("[worker] 閒置:", msg)
        update_progress("IDLE", 0, 0, status="IDLE", log_msg=msg, limit=0)
        return

    # 移除 flag
    try:
        os.remove(FLAG_PATH)
    except Exception:
        pass

    # 等待 config (最多 60 秒)
    update_progress("WAIT_CONFIG", 0, 0, status="RUNNING", log_msg="等待 config.json...", limit=0)
    waited = 0
    while not os.path.exists(CONFIG_PATH) and waited < 60:
        time.sleep(1)
        waited += 1
        if waited % 10 == 0:
            update_progress("WAIT_CONFIG", 0, 0, status="RUNNING", log_msg=f"等待 config.json... ({waited}/60)", limit=0)

    if not os.path.exists(CONFIG_PATH):
        msg = "找不到 config.json"
        print("[worker] 錯誤:", msg)
        update_progress("ERROR", 0, 0, status="IDLE", log_msg=msg, limit=0)
        return

    with open(CONFIG_PATH, "r", encoding="utf-8") as f:
        cfg = json.load(f)

    # 移除 config 以避免重複使用舊配置
    try:
        os.remove(CONFIG_PATH)
    except Exception:
        pass

    keyword = (cfg.get("keyword") or "Nail Salon").strip()
    mode = (cfg.get("mode") or "州別模式").strip()
    state = (cfg.get("state") or "NY").strip().upper()
    limit = int(cfg.get("limit") or 5)
    zip_code = _clean_text(cfg.get("zip_code") or "")
    radius = int(cfg.get("radius") or 5)
    selected_zips = cfg.get("selected_zips") or []

    print("[worker] 配置:", {"keyword": keyword, "mode": mode, "state": state, "limit": limit, "zip_code": zip_code, "radius": radius, "selected_zips_n": len(selected_zips)})

    if not os.path.exists(ZIPS_PATH):
        msg = "找不到 uszips.csv"
        print("[worker] 錯誤:", msg)
        update_progress("ERROR", 0, 0, status="IDLE", log_msg=msg, limit=0)
        return

    # 載入郵政編碼數據
    zdf = pd.read_csv(ZIPS_PATH, dtype={"zip": "string"})
    if "zip" not in zdf.columns:
        # 有些版本稱為 "zipcode"
        if "zipcode" in zdf.columns:
            zdf["zip"] = zdf["zipcode"]
        else:
            raise RuntimeError("uszips.csv 缺少 zip 欄位")
    zdf["zip"] = zdf["zip"].apply(normalize_zip)
    zdf["state_id"] = zdf["state_id"].astype(str).str.strip().str.upper()

    # 目標郵政編碼
    if mode == "單一郵遞區號模式":
        z = normalize_zip(zip_code)
        if not z or len(z) != 5:
            msg = f"無效的郵政編碼: {zip_code}"
            print("[worker] 錯誤:", msg)
            update_progress("ERROR", 0, 0, status="IDLE", log_msg=msg, limit=0)
            return

        # 根據郵政編碼查找行
        targets = zdf[zdf["zip"] == z].copy()
        if targets.empty:
            msg = f"在 uszips.csv 中找不到郵政編碼: {z}"
            print("[worker] 錯誤:", msg)
            update_progress("ERROR", 0, 0, status="IDLE", log_msg=msg, limit=0)
            return

        targets = targets.head(1).copy()
        state = str(targets.iloc[0]["state_id"]).strip().upper()
        print("[worker] 單一郵政編碼:", z, "州:", state)

    else:
        # 州模式
        if selected_zips:
            sel = [normalize_zip(x) for x in selected_zips]
            sel = [x for x in sel if x and len(x) == 5]
            if sel:
                targets = zdf[zdf["zip"].isin(sel)].copy()
            else:
                targets = zdf[zdf["state_id"] == state].head(limit).copy()
        else:
            targets = zdf[zdf["state_id"] == state].head(limit).copy()

        if targets.empty:
            msg = f"沒有找到該州的郵政編碼: {state}"
            print("[worker] 錯誤:", msg)
            update_progress("ERROR", 0, 0, status="IDLE", log_msg=msg, limit=0)
            return

    targets = targets.reset_index(drop=True)
    total_targets = len(targets)
    update_progress("READY", 0, 0, status="RUNNING", log_msg=f"開始掃描 {total_targets} 個郵政編碼", limit=total_targets)

    # 載入現有線索以去重 (名稱+郵政編碼)
    existing_keys = set()
    if os.path.exists(LEADS_CSV):
        try:
            edf = pd.read_csv(LEADS_CSV)
            if "Name" in edf.columns and "Zip" in edf.columns:
                existing_keys = set(zip(edf["Name"].astype(str).str.strip(), edf["Zip"].astype(str).apply(normalize_zip)))
            print("[worker] 已載入現有線索:", len(existing_keys))
        except Exception as e:
            print("[worker] 讀取現有線索失敗:", e)

    # Chrome 設置
    options = Options()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-software-rasterizer")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)

    print("[worker] 啟動 Chrome...")
    driver = gs.Chrome(options=options)

    stealth(
        driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
    )

    leads = []
    scanned = 0
    total_new_leads = 0

    try:
        for idx, r in targets.iterrows():
            scanned += 1
            zip_c = normalize_zip(r["zip"])
            city = _clean_text(r.get("city", ""))
            st = _clean_text(r.get("state_id", state)).upper() or state

            update_progress(zip_c, len(existing_keys) + len(leads), scanned,
                            status="RUNNING",
                            log_msg=f"掃描 {city}, {zip_c} ({scanned}/{total_targets})",
                            limit=total_targets)

            encoded = urllib.parse.quote(keyword)
            if mode == "單一郵遞區號模式" and radius > 0:
                url = f"https://www.google.com/maps/search/{encoded}+near+{zip_c}"
            else:
                url = f"https://www.google.com/maps/search/{encoded}+{zip_c}"

            print(f"[worker] GET: {url}")
            driver.get(url)
            time.sleep(random.uniform(6.5, 9.5))
            handle_consent_if_any(driver)

            # 等待結果區域 (盡力而為)
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='main'], .m6QErb.DxyBCb.kA9KIf.dS8AEf, div.Nv2PK"))
                )
            except Exception:
                print("[worker] 等待結果超時，繼續執行...")

            # 抓取結果卡片
            cards = driver.find_elements(By.CSS_SELECTOR, "div.Nv2PK")
            if not cards:
                # 備用選擇器
                cards = driver.find_elements(By.CSS_SELECTOR, "div[role='article']")
                if not cards:
                    cards = driver.find_elements(By.CSS_SELECTOR, "a[href*='/maps/place/']")

            print(f"[worker] 找到 {len(cards)} 個結果卡片")

            new_in_zip = 0
            for card_idx, card in enumerate(cards[:20]):  # 限制每個郵編處理20個結果
                try:
                    print(f"[worker] 處理卡片 {card_idx+1}/{min(len(cards), 20)}")

                    # 尋找錨點鏈接
                    try:
                        anchor = card.find_element(By.CSS_SELECTOR, "a.hfpxzc")
                    except Exception:
                        try:
                            # 備用：任何指向地點的鏈接
                            links = card.find_elements(By.CSS_SELECTOR, "a[href*='/maps/place/']")
                            anchor = links[0] if links else None
                        except Exception:
                            anchor = None

                    if anchor is None:
                        print(f"[worker] 卡片 {card_idx+1} 沒有找到鏈接，跳過")
                        continue

                    info = extract_business_info(driver, anchor, card_text=(card.text or ""))
                    if not info:
                        print(f"[worker] 卡片 {card_idx+1} 提取信息失敗，跳過")
                        continue

                    name = _clean_text(info.get("Name", ""))
                    if len(name) < 2:
                        print(f"[worker] 卡片 {card_idx+1} 名稱太短，跳過: {name}")
                        continue

                    key = (name.strip(), zip_c)
                    if key in existing_keys:
                        print(f"[worker] 卡片 {card_idx+1} 已存在，跳過: {name}")
                        continue

                    leads.append({
                        "Name": name,
                        "Phone": info.get("Phone", "N/A"),
                        "Address": info.get("Address", "N/A"),
                        "Website": info.get("Website", "N/A"),
                        "City": city,
                        "Zip": zip_c,
                        "State": st,
                        "Latitude": r.get("lat", ""),
                        "Longitude": r.get("lng", ""),
                        "Scraped_Time": now_str(),
                        "Source": "Google Maps",
                        "Search_Keyword": keyword,
                        "Search_Radius": radius if mode == "單一郵遞區號模式" else "",
                        "Scan_Mode": mode,
                    })

                    existing_keys.add(key)
                    new_in_zip += 1
                    total_new_leads += 1

                    # 定期保存
                    if len(leads) % 3 == 0:
                        pd.DataFrame(leads).to_csv(LEADS_CSV, index=False, encoding="utf-8-sig")
                        update_progress(zip_c, len(existing_keys), scanned,
                                        status="RUNNING",
                                        log_msg=f"已保存 {len(leads)} 條新記錄 (進行中...)",
                                        limit=total_targets)

                    time.sleep(random.uniform(1.5, 2.5))

                except Exception as e:
                    print(f"[worker] 處理卡片 {card_idx+1} 時出錯: {e}")
                    continue

            if leads:
                pd.DataFrame(leads).to_csv(LEADS_CSV, index=False, encoding="utf-8-sig")

            print(f"[worker] 郵政編碼 {zip_c} 完成。本郵編新增={new_in_zip} 總新增={len(leads)}")

            # 在處理下一個郵編前等待
            if idx < len(targets) - 1:
                wait_time = random.uniform(5.5, 10.5)
                print(f"[worker] 等待 {wait_time:.1f} 秒後處理下一個郵編...")
                time.sleep(wait_time)

    except Exception as e:
        print(f"[worker] 主循環出錯: {e}")
    finally:
        try:
            driver.quit()
            print("[worker] Chrome 驅動程序已關閉")
        except Exception:
            pass

    # 導出 Excel
    if leads:
        df_final = pd.DataFrame(leads)
        df_final.to_csv(LEADS_CSV, index=False, encoding="utf-8-sig")
        try:
            df_final.to_excel(LEADS_XLSX, index=False)
            print(f"[worker] 已導出 Excel 文件: {LEADS_XLSX}")
        except Exception as e:
            print("[worker] Excel 導出失敗:", e)

        valid_phones = (df_final["Phone"].fillna("N/A") != "N/A").sum() if "Phone" in df_final.columns else 0
        valid_sites  = (df_final["Website"].fillna("N/A") != "N/A").sum() if "Website" in df_final.columns else 0

        msg = f"完成。總新增={len(df_final)} | 有效電話={valid_phones}/{len(df_final)} | 有效網站={valid_sites}/{len(df_final)}"
        print("[worker]", msg)
        update_progress("DONE", len(existing_keys), scanned, status="IDLE", log_msg=msg, limit=total_targets)
    else:
        msg = "完成。沒有找到新線索。"
        print("[worker]", msg)
        update_progress("DONE", len(existing_keys), scanned, status="IDLE", log_msg=msg, limit=total_targets)

    print("[worker] worker.py 執行完成")


# 注意：這部分很重要 - 只有在直接執行此文件時才運行 main()
if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        err = f"致命錯誤: {e}"
        print("[worker]", err)
        update_progress("ERROR", 0, 0, status="IDLE", log_msg=err, limit=0)
'''

# 寫入 worker.py 文件
with open("/content/worker.py", "w", encoding="utf-8") as f:
    f.write(worker_code)

print("✅ worker.py 已生成：/content/worker.py")
print("注意：這個單元格只寫入文件，不執行 worker.py")

In [ ]:
import os
from textwrap import dedent

BASE_DIR = "/content"
os.chdir(BASE_DIR)
print("✅ cwd =", os.getcwd())

app_code = dedent(r"""
import os
import json
import time
import subprocess
from datetime import datetime

import pandas as pd
import streamlit as st

# ✅ 這裡直接用 prompts.py（你上一格已經做通用版）
from prompts import SalesPrompts, DEFAULT_PROFILE

# -------------------------
# 固定 BASE_DIR
# -------------------------
BASE_DIR = os.environ.get("BASE_DIR", "/content")
os.makedirs(BASE_DIR, exist_ok=True)
os.chdir(BASE_DIR)

FLAG_PATH   = os.path.join(BASE_DIR, "dashboard_started.flag")
CONFIG_PATH = os.path.join(BASE_DIR, "config.json")
PROG_PATH   = os.path.join(BASE_DIR, "scan_progress.json")
LEADS_CSV   = os.path.join(BASE_DIR, "leads.csv")
LEADS_XLSX  = os.path.join(BASE_DIR, "leads.xlsx")
WORKER_PY   = os.path.join(BASE_DIR, "worker.py")
ZIPS_PATH   = os.path.join(BASE_DIR, "uszips.csv")
WORKER_LOG  = os.path.join(BASE_DIR, "worker_run.log")

st.set_page_config(page_title="Google Maps Leads Dashboard", layout="wide")

# -------------------------
# Helpers
# -------------------------
def safe_read_json(path: str):
    try:
        if not os.path.exists(path):
            return None
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception:
        return None

def safe_read_csv(path: str):
    try:
        if not os.path.exists(path):
            return None
        df = pd.read_csv(path, encoding="utf-8-sig")
        # ✅ 常見：多出 Unnamed: 0（索引欄）
        drop_cols = [c for c in df.columns if str(c).startswith("Unnamed")]
        if drop_cols:
            df = df.drop(columns=drop_cols)
        # ✅ 保險：把欄名前後空白去掉
        df.columns = [str(c).strip() for c in df.columns]
        return df
    except Exception:
        return None

def write_flag_and_config(config: dict):
    # 先寫 config 再寫 flag
    with open(CONFIG_PATH, "w", encoding="utf-8") as f:
        json.dump(config, f, ensure_ascii=False, indent=2)
    with open(FLAG_PATH, "w", encoding="utf-8") as f:
        f.write(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

def spawn_worker():
    if not os.path.exists(WORKER_PY):
        st.error(f"❌ 找不到 worker.py：{WORKER_PY}")
        return False

    # 確保 worker.log 存在
    if not os.path.exists(WORKER_LOG):
        with open(WORKER_LOG, "w", encoding="utf-8") as f:
            f.write("Worker log started\n")

    # 檢查是否已經有 worker 在運行
    try:
        import psutil
        for proc in psutil.process_iter(["pid", "name", "cmdline"]):
            if proc.info["cmdline"] and "worker.py" in " ".join(proc.info["cmdline"]):
                st.warning("⚠️ 發現已有 worker 在運行，先終止它...")
                proc.terminate()
                proc.wait(timeout=5)
    except Exception:
        pass

    # 啟動新的 worker
    with open(WORKER_LOG, "a", encoding="utf-8") as logf:
        logf.write("\n" + "=" * 60 + "\n")
        logf.write(f"Worker 啟動時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        logf.write("=" * 60 + "\n")

        process = subprocess.Popen(
            ["python", WORKER_PY],
            cwd=BASE_DIR,
            stdout=logf,
            stderr=logf,
            env={**os.environ, "BASE_DIR": BASE_DIR},
        )

    st.info(f"✅ Worker 已啟動 (PID: {process.pid})")
    return True

@st.cache_data(show_spinner=False)
def load_us_zips():
    if not os.path.exists(ZIPS_PATH):
        return None
    # ✅ 保留 zip 前導 0：全部當字串讀
    df = pd.read_csv(ZIPS_PATH, dtype={"zip": "string", "state_id": "string"})
    df["zip"] = df["zip"].astype("string").str.zfill(5)
    df["state_id"] = df["state_id"].astype(str).str.strip().str.upper()
    df["city"] = df["city"].astype(str).fillna("")
    # 常用欄位
    keep_cols = [c for c in ["zip", "city", "state_id", "lat", "lng"] if c in df.columns]
    df = df[keep_cols].copy()
    return df

def now_str():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# ✅ 讓 Website 在表格內可點：把 N/A / 空字串處理掉，補 https://
def normalize_url(x):
    try:
        s = "" if pd.isna(x) else str(x).strip()
        if not s or s.upper() == "N/A":
            return ""
        if s.startswith("http://") or s.startswith("https://"):
            return s
        return "https://" + s
    except Exception:
        return ""

# -------------------------
# Session state: selected_zips list
# -------------------------
if "selected_zips" not in st.session_state:
    st.session_state.selected_zips = []

# -------------------------
# Sidebar: CompanyProfile UI（即時覆蓋 prompts.py）
# -------------------------
def _split_lines(text: str):
    if text is None:
        return []
    return [x.strip() for x in str(text).split("\n") if x.strip()]

# 產業模板（示範：你可自由加/改）
INDUSTRY_TEMPLATES = {
    "Custom (use sidebar values)": None,
    "Nail / Beauty Equipment (Demo)": {
        "BUSINESS_TYPE": "Factory-direct exporter of professional salon equipment",
        "DEFAULT_PRODUCTS": [
            "High-speed electric nail drills (ceramic / steel bits)",
            "Professional dust collectors & ventilation systems",
            "Pedicure chairs for salons",
            "UV / LED curing lamps & sterilization equipment",
        ],
        "VALUE_PROPOSITIONS": [
            "Factory-direct pricing (save 30–50% vs distributors)",
            "US warehouse stock with fast 3–5 day delivery",
            "Low MOQ for first-time cooperation",
            "OEM / private label support",
            "Lifetime technical support",
        ],
    },
    "HVAC / Home Services": {
        "BUSINESS_TYPE": "Local HVAC service provider (installation, repair, maintenance)",
        "DEFAULT_PRODUCTS": [
            "AC installation / replacement",
            "Heating system maintenance",
            "Emergency repair service",
            "Air quality solutions",
        ],
        "VALUE_PROPOSITIONS": [
            "Fast response time and transparent pricing",
            "Licensed & insured technicians",
            "Warranty on parts and labor",
            "Flexible scheduling",
        ],
    },
    "SaaS / Software": {
        "BUSINESS_TYPE": "SaaS provider helping businesses automate operations and increase revenue",
        "DEFAULT_PRODUCTS": [
            "CRM / pipeline automation",
            "Customer support automation",
            "Analytics dashboard",
            "Integrations (Zapier, Slack, Google Sheets)",
        ],
        "VALUE_PROPOSITIONS": [
            "Free trial available",
            "Quick setup in 1–2 days",
            "Dedicated onboarding support",
            "Proven ROI with case studies",
        ],
    },
}

# -------------------------
# UI
# -------------------------
st.title("🗺️ Google Maps Leads Dashboard (Colab)")

with st.sidebar:
    st.header("⚙️ 設定")

    keyword = st.text_input("關鍵字", value="Nail Salon")
    mode = st.selectbox("模式", ["州別模式", "單一郵遞區號模式"], index=0)

    st.divider()
    st.subheader("✉️ CompanyProfile（即時修改）")

    # 語言（你問為何法文：這裡只是「可選」，不需要就不要選）
    lang = st.selectbox("Email Language", ["English", "Spanish", "French"], index=0)

    # 產業模板
    template_name = st.selectbox("Industry Template", list(INDUSTRY_TEMPLATES.keys()), index=1)
    template = INDUSTRY_TEMPLATES.get(template_name)

    # 預設值來源：prompts.py DEFAULT_PROFILE + 可套用模板
    base_company_name = DEFAULT_PROFILE.COMPANY_NAME
    base_your_name = DEFAULT_PROFILE.YOUR_NAME
    base_phone = DEFAULT_PROFILE.CONTACT_PHONE
    base_email = DEFAULT_PROFILE.CONTACT_EMAIL
    base_web = DEFAULT_PROFILE.COMPANY_WEBSITE
    base_biz = DEFAULT_PROFILE.BUSINESS_TYPE
    base_products = DEFAULT_PROFILE.DEFAULT_PRODUCTS
    base_values = DEFAULT_PROFILE.VALUE_PROPOSITIONS

    if template:
        base_biz = template.get("BUSINESS_TYPE", base_biz)
        base_products = template.get("DEFAULT_PRODUCTS", base_products)
        base_values = template.get("VALUE_PROPOSITIONS", base_values)

    company_name = st.text_input("Company Name", value=base_company_name)
    your_name = st.text_input("Your Name", value=base_your_name)
    contact_phone = st.text_input("Contact Phone", value=base_phone)
    contact_email = st.text_input("Contact Email", value=base_email)
    company_website = st.text_input("Company Website", value=base_web)
    business_type = st.text_area("Business Type (one-line positioning)", value=base_biz, height=80)

    products_text_default = "\n".join(base_products) if isinstance(base_products, list) else ""
    values_text_default = "\n".join(base_values) if isinstance(base_values, list) else ""

    ui_default_products_text = st.text_area("Default Products/Services (1 per line)", value=products_text_default, height=140)
    ui_value_props_text = st.text_area("Value Propositions (1 per line)", value=values_text_default, height=140)

    ui_profile = {
        "LANG": lang,
        "COMPANY_NAME": company_name,
        "YOUR_NAME": your_name,
        "CONTACT_PHONE": contact_phone,
        "CONTACT_EMAIL": contact_email,
        "COMPANY_WEBSITE": company_website,
        "BUSINESS_TYPE": business_type,
        "DEFAULT_PRODUCTS": _split_lines(ui_default_products_text),
        "VALUE_PROPOSITIONS": _split_lines(ui_value_props_text),
    }

    with st.expander("💡 Prompt Templates (from prompts.py)", expanded=False):
        st.code(SalesPrompts.prompt_templates_markdown_ui(ui_profile))

    st.divider()

    zips_df = load_us_zips()

    if mode == "州別模式":
        if zips_df is not None and "state_id" in zips_df.columns:
            states = sorted(zips_df["state_id"].dropna().unique().tolist())
            default_state = "CA" if "CA" in states else (states[0] if states else "NY")
            state = st.selectbox("州 (兩碼)", options=states, index=states.index(default_state) if default_state in states else 0)
        else:
            state = st.text_input("州 (兩碼)", value="CA").strip().upper()

        st.caption("✅ 二階選單：先選州別，再選 Zip 加入清單")

        if zips_df is not None:
            sdf = zips_df[zips_df["state_id"] == state].copy()
            sdf["zip_label"] = sdf["zip"] + " - " + sdf["city"].fillna("")
            zip_options = sdf["zip_label"].head(500).tolist()
            chosen = st.selectbox("Zip (下拉選 1 個加入清單)", options=[""] + zip_options, index=0)
            add_btn = st.button("➕ 加入 Zip 清單", use_container_width=True, disabled=(chosen == ""))
            if add_btn and chosen:
                zip_code = chosen.split(" - ")[0].strip()
                if zip_code not in st.session_state.selected_zips:
                    st.session_state.selected_zips.append(zip_code)
                    st.success(f"已加入 {zip_code}")
                    time.sleep(0.3)
                    st.rerun()
        else:
            st.warning("找不到 uszips.csv，暫時無法用二階選單。請先把 uszips.csv 放到 /content")

        st.write("已選 Zip 清單：")
        st.session_state.selected_zips = sorted(list(set(st.session_state.selected_zips)))
        if st.session_state.selected_zips:
            for i, zip_item in enumerate(st.session_state.selected_zips):
                colA, colB = st.columns([4, 1])
                with colA:
                    st.code(zip_item)
                with colB:
                    if st.button("❌", key=f"remove_{i}", help="移除"):
                        st.session_state.selected_zips.remove(zip_item)
                        st.rerun()
        else:
            st.code("(尚未加入)")

        colx, coly = st.columns(2)
        with colx:
            if st.button("🧹 清空 Zip 清單", use_container_width=True):
                st.session_state.selected_zips = []
                st.rerun()
        with coly:
            limit = st.number_input("掃描 zip 數量 (limit)", min_value=1, max_value=200,
                                   value=max(1, len(st.session_state.selected_zips) or 5), step=1)

        selected_zips = st.session_state.selected_zips[:]
        zip_code = ""
        radius = 5

    else:
        state = ""
        limit = 1
        zip_code = st.text_input("郵遞區號 (5碼)", value="10001").strip()
        radius = st.number_input("半徑 (英哩)", min_value=0, max_value=50, value=5, step=1)
        selected_zips = []

    st.divider()

    col1, col2 = st.columns(2)
    with col1:
        start_btn = st.button("🚀 開始爬取", type="primary", use_container_width=True)
    with col2:
        refresh_btn = st.button("🔄 刷新", use_container_width=True)

# -------------------------
# Start action
# -------------------------
if start_btn:
    if mode == "州別模式" and not selected_zips:
        st.warning("請先選擇至少一個 Zip 碼！")
    else:
        cfg = {
            "keyword": keyword,
            "mode": mode,
            "state": state,
            "limit": int(limit),
            "zip_code": zip_code,
            "radius": int(radius),
            "selected_zips": selected_zips,
            "created_at": now_str(),
        }
        write_flag_and_config(cfg)
        ok = spawn_worker()
        if ok:
            st.success("✅ 已送出任務！Worker 已在背景啟動。請看右側進度與下方 leads。")
        else:
            st.error("❌ Worker 啟動失敗，請檢查 /content 是否有 worker.py")

if refresh_btn:
    st.rerun()

# -------------------------
# Main panels
# -------------------------
col1, col2 = st.columns([1, 1])

with col1:
    st.subheader("📈 進度狀態")
    prog = safe_read_json(PROG_PATH)
    if prog:
        st.json(prog, expanded=False)
        status = prog.get("status", "")
        limit_n = prog.get("limit", 0) or 0
        scanned = prog.get("total_scanned", 0) or 0
        found = prog.get("found_count", 0) or 0

        if isinstance(limit_n, int) and limit_n > 0:
            progress_value = min(1.0, scanned / max(1, int(limit_n)))
            st.progress(progress_value)
            st.metric("進度", f"{scanned}/{limit_n} ({progress_value*100:.1f}%)")
        st.metric("已掃描", scanned)
        st.metric("已找到", found)

        status_color = {
            "IDLE": "gray",
            "RUNNING": "blue",
            "DONE": "green",
            "ERROR": "red",
            "WAIT_CONFIG": "orange"
        }.get(status, "gray")

        st.markdown(f"**狀態**: <span style='color:{status_color}'>{status}</span>", unsafe_allow_html=True)
        st.caption(prog.get("log", ""))
    else:
        st.info("尚無進度檔 scan_progress.json（按『開始爬取』後才會出現）")

with col2:
    st.subheader("📄 Worker 日誌 (最新)")
    if os.path.exists(WORKER_LOG):
        try:
            with open(WORKER_LOG, "r", encoding="utf-8", errors="ignore") as f:
                lines = f.readlines()[-100:]
            st.code("".join(lines))

            with open(WORKER_LOG, "rb") as f:
                st.download_button(
                    label="⬇️ 下載日誌",
                    data=f,
                    file_name="worker_run.log",
                    mime="text/plain",
                    use_container_width=True
                )
        except Exception as e:
            st.error(f"讀取日誌失敗: {e}")
    else:
        st.caption("尚無 worker_run.log")

st.divider()

st.subheader("🧾 Leads 數據")
df = safe_read_csv(LEADS_CSV)
if df is None or df.empty:
    st.warning("尚無 leads.csv（worker 跑完或跑到一部分才會產生）")
else:
    st.success(f"✅ leads.csv 共 {len(df)} 筆記錄")

    # ✅ 讓表格 Website 可點：只處理顯示用，不改原檔
    if "Website" in df.columns:
        df["Website"] = df["Website"].apply(normalize_url)

    # 顯示統計信息
    colA, colB, colC = st.columns(3)
    with colA:
        valid_phones = (df["Phone"].fillna("N/A") != "N/A").sum() if "Phone" in df.columns else 0
        st.metric("有效電話", f"{valid_phones}/{len(df)}")
    with colB:
        valid_sites = (df["Website"].fillna("") != "").sum() if "Website" in df.columns else 0
        st.metric("有效網站", f"{valid_sites}/{len(df)}")
    with colC:
        unique_zips = df["Zip"].nunique() if "Zip" in df.columns else 0
        st.metric("唯一郵編", unique_zips)

    # ✅ 表格內網址可點（新分頁打開）
    col_cfg = {}
    if "Website" in df.columns:
        col_cfg["Website"] = st.column_config.LinkColumn(
            "Website",
            display_text="🔗 Open",
            help="Click to open in a new tab",
        )

    st.dataframe(
        df,
        use_container_width=True,
        height=300,
        column_config=col_cfg if col_cfg else None
    )

    # -------------------------
    # 詳細信息與操作按鈕
    # -------------------------
    st.subheader("🔍 詳細操作")

    if len(df) > 0:
        business_options = df["Name"].tolist() if "Name" in df.columns else [str(i) for i in range(len(df))]
        selected_business_index = st.selectbox(
            "選擇商家進行操作:",
            range(len(business_options)),
            format_func=lambda x: f"{business_options[x]} ({df.iloc[x]['City']}, {df.iloc[x]['State']})"
            if ("City" in df.columns and "State" in df.columns) else business_options[x]
        )

        if selected_business_index is not None:
            selected_business = df.iloc[selected_business_index]

            with st.expander(f"📋 {selected_business.get('Name','(no name)')} 詳細信息", expanded=True):
                colL, colR = st.columns(2)
                with colL:
                    st.markdown(f"**商家名稱:** {selected_business.get('Name','')}")
                    st.markdown(f"**電話:** {selected_business.get('Phone','')}")
                    clean_address = str(selected_business.get("Address","")).replace("", "").replace("\n", " ").strip()
                    st.markdown(f"**地址:** {clean_address}")
                with colR:
                    st.markdown(f"**城市:** {selected_business.get('City','')}")
                    st.markdown(f"**郵編:** {selected_business.get('Zip','')}")
                    st.markdown(f"**州:** {selected_business.get('State','')}")
                    website = str(selected_business.get("Website","")).strip()
                    if website:
                        st.markdown(f"**網站:** [{website}]({website})")

            st.markdown("### 🛠️ 快速操作")
            col1, col2, col3, col4 = st.columns(4)

            name = str(selected_business.get("Name","")).strip()
            city = str(selected_business.get("City","")).strip()
            state = str(selected_business.get("State","")).strip()

            with col1:
                google_url = f"https://www.google.com/search?q={name.replace(' ', '+')}+{city.replace(' ', '+')}+{state}"
                st.markdown(
                    f'<a href="{google_url}" target="_blank"><button style="background-color: #4285F4; color: white; padding: 10px 15px; border: none; border-radius: 5px; cursor: pointer;">🔍 Google</button></a>',
                    unsafe_allow_html=True
                )

            with col2:
                yelp_url = f"https://www.yelp.com/search?find_desc={name.replace(' ', '+')}&find_loc={city.replace(' ', '+')}+{state}"
                st.markdown(
                    f'<a href="{yelp_url}" target="_blank"><button style="background-color: #D32323; color: white; padding: 10px 15px; border: none; border-radius: 5px; cursor: pointer;">⭐ Yelp</button></a>',
                    unsafe_allow_html=True
                )

            with col3:
                ig_url = f"https://www.instagram.com/explore/search/keyword/?q={name.replace(' ', '%20')}%20{city.replace(' ', '%20')}"
                st.markdown(
                    f'<a href="{ig_url}" target="_blank"><button style="background: linear-gradient(45deg, #405DE6, #5851DB, #833AB4, #C13584, #E1306C, #FD1D1D); color: white; padding: 10px 15px; border: none; border-radius: 5px; cursor: pointer;">📷 IG</button></a>',
                    unsafe_allow_html=True
                )

            with col4:
                fb_url = f"https://www.facebook.com/search/top?q={name.replace(' ', '%20')}%20{city.replace(' ', '%20')}"
                st.markdown(
                    f'<a href="{fb_url}" target="_blank"><button style="background-color: #1877F2; color: white; padding: 10px 15px; border: none; border-radius: 5px; cursor: pointer;">👥 FB</button></a>',
                    unsafe_allow_html=True
                )

            st.divider()

            st.subheader("📧 生成開發信（使用 prompts.py + Sidebar UI）")

            st.markdown("**(可選) 這封信要用的 Products/Services（每行一個；留空則用 Sidebar 的 Default Products）**")
            products_text = st.text_area(
                "Email Products/Services (optional):",
                value="",
                height=120,
                help="每行一個；留空會使用 Sidebar 的 Default Products"
            )
            products_override = _split_lines(products_text) if products_text.strip() else None

            if st.button("🤖 生成開發信", type="primary", use_container_width=True):
                email_content = SalesPrompts.generate_sales_email_ui(
                    business_name=name,
                    city=city,
                    state=state,
                    products=products_override,
                    ui_profile=ui_profile,
                )

                st.markdown("### 📝 生成的開發信:")
                st.code(email_content)

                st.text_area(
                    "開發信內容 (可複製):",
                    value=email_content,
                    height=260,
                    key="email_content_area"
                )

                st.download_button(
                    label="⬇️ 下載開發信 (.txt)",
                    data=email_content,
                    file_name=f"sales_email_{name.replace(' ', '_')}.txt",
                    mime="text/plain",
                    use_container_width=True
                )

    else:
        st.info("沒有可用的商家數據")

    st.divider()
    st.subheader("💾 下載數據")

    csv_bytes = df.to_csv(index=False).encode("utf-8-sig")
    st.download_button("⬇️ 下載 CSV", data=csv_bytes, file_name="leads.csv", mime="text/csv", use_container_width=True)

    if os.path.exists(LEADS_XLSX):
        with open(LEADS_XLSX, "rb") as f:
            st.download_button(
                "⬇️ 下載 Excel",
                data=f,
                file_name="leads.xlsx",
                mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
                use_container_width=True
            )
    else:
        try:
            import io
            buf = io.BytesIO()
            df.to_excel(buf, index=False)
            st.download_button(
                "⬇️ 下載 Excel (即時生成)",
                data=buf.getvalue(),
                file_name="leads.xlsx",
                mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
                use_container_width=True
            )
        except Exception as e:
            st.warning(f"無法生成 Excel 文件: {e}")

st.caption("💡 提示：在 Colab/ngrok 環境中，建議定期手動點擊『刷新』按鈕查看最新狀態。")
""")

with open("/content/app_lead.py", "w", encoding="utf-8") as f:
    f.write(app_code)

print("✅ app_lead.py 已更新：/content/app_lead.py")
print("✅ 已加入 Sidebar CompanyProfile UI + 語言選擇 + 產業模板下拉")
print("✅ 開發信改為使用 prompts.py 的 SalesPrompts.generate_sales_email_ui()")
print("✅ Leads 表格 Website 欄位維持可點擊（新分頁開啟）")


In [ ]:
import os, re, time, threading, subprocess, signal, sys

# ===========================
# Config
# ===========================
BASE_DIR = "/content"
APP_FILE = "app_lead.py"   # <- 你的 Streamlit 程式
PORT = 8501
STREAMLIT_LOG = os.path.join(BASE_DIR, "streamlit.log")
CLOUDFLARED_LOG = os.path.join(BASE_DIR, "cloudflared.log")

os.makedirs(BASE_DIR, exist_ok=True)
os.chdir(BASE_DIR)
print("✅ cwd =", os.getcwd())

if not os.path.exists(APP_FILE):
    raise FileNotFoundError(f"❌ 找不到 {APP_FILE}，請先把 {APP_FILE} 寫到 /content")

# ===========================
# 1) Kill old processes
# ===========================
!pkill -f "streamlit run" 2>/dev/null || true
!pkill -f "cloudflared tunnel" 2>/dev/null || true
time.sleep(1)

# ===========================
# 2) Start Streamlit (background)
# ===========================
os.environ["STREAMLIT_BROWSER_GATHER_USAGE_STATS"] = "false"
os.environ["STREAMLIT_SERVER_HEADLESS"] = "true"

def run_streamlit():
    cmd = [
        sys.executable, "-m", "streamlit", "run", APP_FILE,
        "--server.port", str(PORT),
        "--server.address", "0.0.0.0",
        "--server.headless", "true",
        "--server.enableCORS", "false",
        "--global.developmentMode", "false",
        "--browser.gatherUsageStats", "false",
    ]
    with open(STREAMLIT_LOG, "w", encoding="utf-8") as f:
        subprocess.Popen(cmd, stdout=f, stderr=f, cwd=BASE_DIR)

threading.Thread(target=run_streamlit, daemon=True).start()
time.sleep(2)
print(f"🚀 streamlit started (local): http://localhost:{PORT}")
print("🧾 streamlit log:", STREAMLIT_LOG)

# ===========================
# 3) Wait until port is open
# ===========================
import socket
def is_port_open(port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(("127.0.0.1", port)) == 0

for _ in range(30):
    if is_port_open(PORT):
        print("✅ Streamlit is up on port", PORT)
        break
    time.sleep(1)
else:
    print("❌ Streamlit 沒起來，印出 streamlit.log 最後 3000 字：")
    with open(STREAMLIT_LOG, "r", encoding="utf-8", errors="ignore") as f:
        print(f.read()[-3000:])
    raise RuntimeError("Streamlit failed to start")

# ===========================
# 4) Install cloudflared if missing
# ===========================
if not os.path.exists("/usr/local/bin/cloudflared"):
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared
    !chmod +x /usr/local/bin/cloudflared
print("✅ cloudflared ready:", "/usr/local/bin/cloudflared")

# ===========================
# 5) Start Cloudflare Quick Tunnel and parse URL
# ===========================
cmd = ["cloudflared", "tunnel", "--url", f"http://localhost:{PORT}", "--no-autoupdate"]
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)

pattern = re.compile(r"https://[a-z0-9-]+\.trycloudflare\.com")
public_url = None

start = time.time()
lines = []

print("\n⏳ Starting Cloudflare Quick Tunnel... (watch logs)")
with open(CLOUDFLARED_LOG, "w", encoding="utf-8") as logf:
    while time.time() - start < 45:
        line = p.stdout.readline()
        if not line:
            continue
        line = line.strip()
        lines.append(line)
        logf.write(line + "\n")
        print(line)

        m = pattern.search(line)
        if m:
            public_url = m.group(0)
            break

if public_url:
    print("\n🌍 PUBLIC URL (Cloudflare Quick Tunnel):")
    print(public_url)
    print("\n✅ 提醒：不要 Stop 這個 cell / 不要重啟 runtime，網址才會保持可用。")
else:
    print("\n❌ Failed to get trycloudflare URL within 45s.")
    print("🧾 cloudflared log:", CLOUDFLARED_LOG)
    print("（你可以把 cloudflared.log 打開看最後幾行）")
